#1.Imports

In [ ]:
# Install the Pandas library ydata_profiling
import sys
!{sys.executable} -m pip install -U ydata-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.7/102.7 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.5/679.5 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.4/455.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27079 sha256=8c2692be050034e8566bff6e8b8a476adfb26f2dfbd8a353838aae38dddbf39f
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin
  Attempting uninstall: wordcloud
    Found existing installation: wordcloud 1.8.2.2
    Uninstalling wordclo

In [ ]:
# System
import os
from google.colab import files
import warnings

import numpy as np
import pandas as pd
import datetime
import json
import plotly.express as px
from sqlalchemy import create_engine, text
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.decomposition import KernelPCA

# Machine learning - Preprocessing
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, QuantileTransformer, PowerTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin


# Machine learning - Automatisation
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import set_config


from ydata_profiling import ProfileReport
from google.colab import files


import matplotlib.pyplot as plt

In [ ]:
# Silence warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Define the connection string
connection_string = "postgresql://co2-sa-db.postgres.database.azure.com:5432/seattlebeb?user=co2sodapg&password=Greta2023&sslmode=require"

# Create a SQLAlchemy engine
engine = create_engine(connection_string)

# Create a connection and execute the query
with engine.connect() as conn:
    query = text("SELECT * FROM buildings")
    data = pd.read_sql(query, conn)

data.head()

,osebuildingid,buildingtype,primarypropertytype,zipcode,taxparcelidentificationnumber,councildistrictcode,neighborhood,latitude,longitude,yearbuilt,...,sourceeuiwn_kbtu_sf,siteenergyuse_kbtu,siteenergyusewn_kbtu,steamuse_kbtu,electricity_kbtu,naturalgas_kbtu,defaultdata,compliancestatus,outlier,totalghgemissions
0,1,NonResidential,Hotel,98101.0,0659000030,7,DOWNTOWN,47.61220,-122.33799,1927,...,189.000000,7226362.5,7456910.0,2003882.00,3946027.0,1276453.0,False,Compliant,None,249.98
1,2,NonResidential,Hotel,98101.0,0659000220,7,DOWNTOWN,47.61317,-122.33393,1996,...,179.399994,8387933.0,8664479.0,0.00,3242851.0,5145082.0,False,Compliant,None,295.86
2,3,NonResidential,Hotel,98101.0,0659000475,7,DOWNTOWN,47.61393,-122.33810,1969,...,244.100006,72587024.0,73937112.0,21566554.00,49526664.0,1493800.0,False,Compliant,None,2089.28
3,5,NonResidential,Hotel,98101.0,0659000640,7,DOWNTOWN,47.61412,-122.33664,1926,...,224.000000,6794584.0,6946800.5,2214446.25,2768924.0,1811213.0,False,Compliant,None,286.43
4,8,NonResidential,Hotel,98121.0,0659000970,7,DOWNTOWN,47.61375,-122.34047,1980,...,215.600006,14172606.0,14656503.0,0.00,5368607.0,8803998.0,False,Compliant,None,505.01


In [ ]:
# Remove IDs
df = data.drop(['osebuildingid'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   buildingtype                     3376 non-null   object 
 1   primarypropertytype              3376 non-null   object 
 2   zipcode                          3360 non-null   float64
 3   taxparcelidentificationnumber    3376 non-null   object 
 4   councildistrictcode              3376 non-null   int64  
 5   neighborhood                     3376 non-null   object 
 6   latitude                         3376 non-null   float64
 7   longitude                        3376 non-null   float64
 8   yearbuilt                        3376 non-null   int64  
 9   numberofbuildings                3368 non-null   float64
 10  numberoffloors                   3376 non-null   int64  
 11  propertygfatotal                 3376 non-null   int64  
 12  propertygfaparking  

#2.Data cleaning

In [ ]:
df['compliancestatus'].value_counts()

Compliant                       3211
Error - Correct Default Data     113
Non-Compliant                     37
Missing Data                      15
Name: compliancestatus, dtype: int64

In [ ]:
# Fix the values for 'compliancestatus'
df['compliancestatus'].replace('Missing Data', np.nan, inplace=True)

In [ ]:
df['neighborhood'].value_counts()

DOWNTOWN                  573
EAST                      453
MAGNOLIA / QUEEN ANNE     423
GREATER DUWAMISH          375
NORTHEAST                 280
LAKE UNION                251
NORTHWEST                 210
SOUTHWEST                 166
NORTH                     145
BALLARD                   126
CENTRAL                   107
SOUTHEAST                  95
DELRIDGE                   80
North                      42
Central                    27
Northwest                  11
Ballard                     7
Delridge                    4
DELRIDGE NEIGHBORHOODS      1
Name: neighborhood, dtype: int64

In [ ]:
# Fix the case and replace duplicates
df['neighborhood'] = df['neighborhood'].str.upper()
df['neighborhood'] = df['neighborhood'].str.replace('DELRIDGE NEIGHBORHOODS','DELRIDGE')
df['neighborhood'].value_counts()

DOWNTOWN                 573
EAST                     453
MAGNOLIA / QUEEN ANNE    423
GREATER DUWAMISH         375
NORTHEAST                280
LAKE UNION               251
NORTHWEST                221
NORTH                    187
SOUTHWEST                166
CENTRAL                  134
BALLARD                  133
SOUTHEAST                 95
DELRIDGE                  85
Name: neighborhood, dtype: int64

In [ ]:
# Drop all rows where the 'outlier' column is not null
df.drop(index=df[df['outlier'].notna()].index, inplace=True)
df.drop(['outlier'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 0 to 3375
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   buildingtype                     3344 non-null   object 
 1   primarypropertytype              3344 non-null   object 
 2   zipcode                          3328 non-null   float64
 3   taxparcelidentificationnumber    3344 non-null   object 
 4   councildistrictcode              3344 non-null   int64  
 5   neighborhood                     3344 non-null   object 
 6   latitude                         3344 non-null   float64
 7   longitude                        3344 non-null   float64
 8   yearbuilt                        3344 non-null   int64  
 9   numberofbuildings                3336 non-null   float64
 10  numberoffloors                   3344 non-null   int64  
 11  propertygfatotal                 3344 non-null   int64  
 12  propertygfaparking  

In [ ]:
# Fix 'energystarscore'
df['energystarscore'] = df['energystarscore'].replace("NULL", None)

In [ ]:
# Drop columns with too many missing values
threshold = 70 # minimum percentage of non-null cells in each column
for column in df.columns:
  if df[column].isnull().sum() / len(df) *100 > (100 - threshold):
      df.drop([column], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 0 to 3375
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3344 non-null   object 
 1   primarypropertytype            3344 non-null   object 
 2   zipcode                        3328 non-null   float64
 3   taxparcelidentificationnumber  3344 non-null   object 
 4   councildistrictcode            3344 non-null   int64  
 5   neighborhood                   3344 non-null   object 
 6   latitude                       3344 non-null   float64
 7   longitude                      3344 non-null   float64
 8   yearbuilt                      3344 non-null   int64  
 9   numberofbuildings              3336 non-null   float64
 10  numberoffloors                 3344 non-null   int64  
 11  propertygfatotal               3344 non-null   int64  
 12  propertygfaparking             3344 non-null   i

In [ ]:
# Impute 'zipcode' missing values from 'latitude' and 'longitude'
location = ['zipcode', 'latitude', 'longitude']
dfl = df[location]
dfl = pd.DataFrame(data=KNNImputer(n_neighbors=10).fit_transform(dfl), index=df.index, columns=location)
df = df.drop(columns=location).join(dfl)

#x.Feature Engineering

In [ ]:
# Derive the buildings age from 'yearbuilt'
current_year = datetime.datetime.now().year
df['age'] = df['yearbuilt'].apply(lambda x: current_year - x)
df.drop(['yearbuilt'], axis=1, inplace=True)

According to the brief ("les relevés sont coûteux à obtenir"), the city of Seattle would like to predict CO2 emissions without knowing the energy consumption, so we are only going to keep the type of energy usage for each building :

In [ ]:
# Create new columns for the types of energy consumption (steam/electricity/gaz)df['numberofbuildings'] = df['numberofbuildings'].astype('int64')
def energy_usage(cell):
    if cell > 0:
      return True
    else:
      return False
df['steamuse_kbtu'] = df['steamuse_kbtu'].apply(energy_usage)
df['electricity_kbtu'] = df['electricity_kbtu'].apply(energy_usage)
df['naturalgas_kbtu'] = df['naturalgas_kbtu'].apply(energy_usage)
df.rename(columns={'steamuse_kbtu': 'steam'}, inplace=True)
df.rename(columns={'electricity_kbtu': 'electricity'}, inplace=True)
df.rename(columns={'naturalgas_kbtu': 'naturalgas'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 0 to 3375
Data columns (total 30 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3344 non-null   object 
 1   primarypropertytype            3344 non-null   object 
 2   taxparcelidentificationnumber  3344 non-null   object 
 3   councildistrictcode            3344 non-null   int64  
 4   neighborhood                   3344 non-null   object 
 5   numberofbuildings              3336 non-null   float64
 6   numberoffloors                 3344 non-null   int64  
 7   propertygfatotal               3344 non-null   int64  
 8   propertygfaparking             3344 non-null   int64  
 9   propertygfabuilding_s          3344 non-null   int64  
 10  listofallpropertyusetypes      3335 non-null   object 
 11  largestpropertyusetype         3335 non-null   object 
 12  largestpropertyusetypegfa      3324 non-null   f

#x.Target engineering

'sourceeuiwn_kbtu_sf', 'sourceeui_kbtu_sf', 'siteeuiwn_kbtu_sf', 'siteeui_kbtu_sf', 'siteenergyusewn_kbtu', 'siteenergyuse_kbtu', 'totalghgemissions'

In [ ]:
df['source_site'] = df['sourceeuiwn_kbtu_sf'] / df['siteeuiwn_kbtu_sf']
df['source_site'].describe()

count    3305.000000
mean        2.555850
std         0.628898
min        -0.420000
25%         2.115583
50%         2.634804
75%         3.138461
max        13.211111
Name: source_site, dtype: float64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3344 entries, 0 to 3375
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3344 non-null   object 
 1   primarypropertytype            3344 non-null   object 
 2   taxparcelidentificationnumber  3344 non-null   object 
 3   councildistrictcode            3344 non-null   int64  
 4   neighborhood                   3344 non-null   object 
 5   numberofbuildings              3336 non-null   float64
 6   numberoffloors                 3344 non-null   int64  
 7   propertygfatotal               3344 non-null   int64  
 8   propertygfaparking             3344 non-null   int64  
 9   propertygfabuilding_s          3344 non-null   int64  
 10  listofallpropertyusetypes      3335 non-null   object 
 11  largestpropertyusetype         3335 non-null   object 
 12  largestpropertyusetypegfa      3324 non-null   f

In [ ]:
# Remove the few rows where target values are missing
df.dropna(subset=['source_site'], inplace=True)
df.dropna(subset=['siteenergyuse_kbtu'], inplace=True)
df.dropna(subset=['siteenergyusewn_kbtu'], inplace=True)
df.dropna(subset=['totalghgemissions'], inplace=True)
df.dropna(subset=['siteeui_kbtu_sf'], inplace=True)
df.dropna(subset=['siteeuiwn_kbtu_sf'], inplace=True)
df.dropna(subset=['sourceeui_kbtu_sf'], inplace=True)
df.dropna(subset=['sourceeuiwn_kbtu_sf'], inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3305 entries, 0 to 3375
Data columns (total 31 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   buildingtype                   3305 non-null   object 
 1   primarypropertytype            3305 non-null   object 
 2   taxparcelidentificationnumber  3305 non-null   object 
 3   councildistrictcode            3305 non-null   int64  
 4   neighborhood                   3305 non-null   object 
 5   numberofbuildings              3305 non-null   float64
 6   numberoffloors                 3305 non-null   int64  
 7   propertygfatotal               3305 non-null   int64  
 8   propertygfaparking             3305 non-null   int64  
 9   propertygfabuilding_s          3305 non-null   int64  
 10  listofallpropertyusetypes      3305 non-null   object 
 11  largestpropertyusetype         3305 non-null   object 
 12  largestpropertyusetypegfa      3294 non-null   f

In [ ]:
# Now that there are less missing values, change the feature types
df['zipcode'] = df['zipcode'].astype('object')
df['councildistrictcode'] = df['councildistrictcode'].astype('object')
df['numberofbuildings'] = df['numberofbuildings'].astype('int64')
#df['largestpropertyusetypegfa'] = df['largestpropertyusetypegfa'].astype('int64')
df.dtypes

buildingtype                      object
primarypropertytype               object
taxparcelidentificationnumber     object
councildistrictcode               object
neighborhood                      object
numberofbuildings                  int64
numberoffloors                     int64
propertygfatotal                   int64
propertygfaparking                 int64
propertygfabuilding_s              int64
listofallpropertyusetypes         object
largestpropertyusetype            object
largestpropertyusetypegfa        float64
energystarscore                   object
siteeui_kbtu_sf                  float64
siteeuiwn_kbtu_sf                float64
sourceeui_kbtu_sf                float64
sourceeuiwn_kbtu_sf              float64
siteenergyuse_kbtu               float64
siteenergyusewn_kbtu             float64
steam                               bool
electricity                         bool
naturalgas                          bool
defaultdata                         bool
compliancestatus

#3.Data Profiling

In [ ]:
# Fix for the Google Colab bug "ValueError: Only supported for TrueType fonts" : the report gets generated externally by the profiling.py file
df.to_csv('co2_eda.csv', index=False)
files.download('co2_eda.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

L'intégralité du rapport est disponible sur https://seabeb.azurewebsites.net/dashboard (ou http://127.0.0.1:8000/dashboard si la web app est arrêtée).

##### Variable dépendante : TotalGHGEmissions
L'immense majorité des immeubles émet moins de 50 tonnes, mais on peut noter la présence de quelques outliers qui polluent énormément.
La rubrique "Interactions" montre que les émissions tendent à croître avec le nombre d'étages, mais aussi avec le score Energy Star, ce qui est beaucoup moins logique : les immeubles les mieux notés devraient au contraire émettre moins de gaz à effet de serre...

La superficie des immeubles (en premier lieu, 'propertygfatotal') est fortement liée au niveau d'émission, comme l'indique la matrice de corrélation.

#4.Sélection des features

Idéalement, cette tâche devrait être effectuée autoùatiquement avec [RFECV](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFECV.html), mais l'algorithme requiert des ressources trop élevées pour Google Colab.

L'autre solution serait d'entraîner un modèle comprenant toutes les features avec XGBoost et sa régularisation L1 : contrairement aux modèles comme Lasso ou ElasticNet, il peut gérer la non-linéarité. Le OneHotEncoder pourrait être remplacé avec un TargetEncoder qui diminuerait grandement le nombre de colomnes à traiter. Malheureusement, aucun Target encoder testé ne semble fonctionner correctement dans le cas de targets multiples au sein d'une pipeline.

A défaut, nous allons eefectuer une première sélection sur des bases staistiques, puis XGBoost nous aidera


##4.1Statistical

La variable 'electricity' peut être négligée parce qu'elle est fortement corrélée à la nouvelle feature 'source_site' et présente un grand déséquilibre : seulement 15 immeubles ne consomment pas d'électricité.

La variable 'primarypropertytype' sera préférée à 'buildingtype' parce qu'elle est davantage corrélée avec les targets.
Pour la même raison, nous choisirons 'councildistrictcode' parmi toutes les variables de localisation qui sont plus ou moins corrélées. Et 'propertygfatotal' à la place de 'propertygfabuilding_s' et 'largestpropertyusetypegfa'.


Après le nettoyage des données, il s'avère que la variable 'compliancestatus' est devenue constante sur l'ensemble des lignes restantes, donc on peut la supprimer également. Le processus de sélection des features sera approfondi ultérieurement, mais voici le dataframe qui servira de base à la modélisation :

In [ ]:
# Drop unnecessary features
df_stats = df.drop(['electricity', 'buildingtype', 'propertygfabuilding_s', 'largestpropertyusetypegfa', 'neighborhood', 'zipcode', 'latitude', 'longitude', 'compliancestatus', 'defaultdata', 'taxparcelidentificationnumber'], axis=1)
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3334 entries, 0 to 3375
Data columns (total 16 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   primarypropertytype        3334 non-null   object 
 1   councildistrictcode        3334 non-null   object 
 2   numberofbuildings          3334 non-null   int64  
 3   numberoffloors             3334 non-null   int64  
 4   propertygfatotal           3334 non-null   int64  
 5   propertygfaparking         3334 non-null   int64  
 6   listofallpropertyusetypes  3334 non-null   object 
 7   largestpropertyusetype     3334 non-null   object 
 8   energystarscore            2509 non-null   float64
 9   siteenergyuse_kbtu         3334 non-null   float64
 10  siteenergyusewn_kbtu       3334 non-null   float64
 11  steam                      3334 non-null   int64  
 12  naturalgas                 3334 non-null   int64  
 13  totalghgemissions          3334 non-null   float

In [ ]:
# CSV export
df_stats.to_csv('co2_stats.csv', index=False)
files.download('co2_stats.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#KernelPCA loss of 0.5 of R2

#4.Recommandations

In [ ]:
fig = px.bar(df, x="neighborhood", y="totalghgemissions", title="Aménagement des espaces verts", color_discrete_sequence=['green'])
fig.show()
fig.write_html('neighborhood.html')
files.download('neighborhood.html')

En supplément du centre-ville, il serait souhaitable de privilégier la partie est de la ville (Northeast+East).

In [ ]:
fig = px.scatter(df, x='age', y='totalghgemissions', size='totalghgemissions', color_discrete_sequence=['red'], title="Adaptation des politiques de réhabilitation des immeubles")
fig.show()
fig.write_html('age.html')
files.download('age.html')

Outre les immeubles très anciens, ceux qui ont entre 20 et 40 ans pourraient être intégrés dans les politiques de subventionnement des travaux.

In [ ]:
fig = px.scatter(df, x='energystarscore', y='totalghgemissions', size='totalghgemissions', color_discrete_sequence=['blue'], title="Nécessité d'un nouveau label Energy Star")
fig.show()
fig.write_html('estar.html')
files.download('estar.html')

L'absence de corrélation entre les émissions de CO2 et le score Energy Star semble justifier la création d'un nouveau label, tel qu'il a été envisagé dans ce papier : https://www.researchgate.net/publication/342831494_EnergyStar_Towards_more_accurate_and_explanatory_building_energy_benchmarking